This code replicates methods from @kaandorp_optimal_2007.

In [ ]:
import logging
import copy
import datetime
import numpy as np
from scipy.stats import poisson
from scipy.stats import lognorm
from scipy import signal
from scipy.special import comb
import plotly.graph_objs as go
import plotly.offline as pyo
from itertools import chain, combinations
import copy

Schedule $x$ is a vector with each element representing the number of scheduled patients.

In [ ]:
#x = [2, 1, 0, 2]
x = [2, 0, 0, 0, 2]

The number of possible schedules is

$$\dbinom{N+T-1}{N}$$

In [ ]:
N = sum(x)
T = len(x)
result = comb(N + T - 1, N, exact=True)  # exact=True returns the exact integer
print(result) 

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return [list(item) for item in chain.from_iterable(combinations(s, r) for r in range(len(s)+1))]

In [ ]:
test = powerset([1,2,3])
list(test)

Define the vectors

$$
\left\{
\begin{array}{c}
\vec{u_1}, \\
\vec{u_2}, \\
\vec{u_3}, \\
\vdots \\
\vec{u_{T-1}}, \\
\vec{u_T} \\
\end{array}
\right\} = 
\left\{
\begin{array}{c}
(-1, 0,...., 0, 1), \\
(1, -1, 0,...., 0), \\
(0, 1, -1,...., 0), \\
\vdots \\
(0,...., 1, -1, 0), \\
(0,...., 0, 1, -1) \\
\end{array}
\right\}
$$ and take $V^* = \{u_1,....,u_T\}$.

In [ ]:
def get_v_star(t):
    # Create an initial vector 'u' of zeros with length 't'
    u = np.zeros(t)
    # Set the first element of vector 'u' to -1
    u[0] = -1
    # Set the last element of vector 'u' to 1
    u[-1] = 1
    # Initialize the list 'v_star' with the initial vector 'u'
    v_star = [u]
    # Loop over the length of 'u' minus one times
    for i in range(len(u) - 1):
        # Append the last element of 'u' to the front of 'u'
        u = np.append(u[-1], u)
        # Remove the last element of 'u' to maintain the same length
        u = np.delete(u, -1)
        # Append the updated vector 'u' to the list 'v_star'
        v_star.append(u)
    # Convert the list of vectors 'v_star' into a NumPy array and return it
    return(np.array(v_star))

# Example of function call:
# This will create a 4x4 matrix where each row is a cyclically shifted version of the first row
get_v_star(4)

As the neighborhood of schedule $x$ we take all vectors of the form $x + v_1+..+v_k$ with $v_1,....,v_k \in V^*$

In [ ]:
# Set the value of 't'
t = len(x)

# Generate a matrix 'v_star' using the 'get_v_star' function
v_star = get_v_star(t)

# Generate all possible non-empty subsets (powerset) of the set {0, 1, 2, ..., t-1}
# 'ids' will be a list of tuples, where each tuple is a subset of indices
ids = list(powerset(range(t)))

# Select the vectors from 'v_star' that correspond to the indices in each subset
# 'sub_sets' will be a list of lists, where each inner list contains vectors from 'v_star'
sub_sets = [v_star[i] for i in ids]

# Sum the vectors within each subset and flatten the result to get a 1-D array
# 'summed_sets' will be a list of 1-D numpy arrays, where each array is the sum of vectors
summed_sets = [np.sum(sub_sets[i], axis=0).flatten() for i in range(len(sub_sets))]

# 'summed_sets' is the final output
summed_sets

In [ ]:
y = np.array([x + summed_sets[i] for i in range(len(summed_sets))])
y

such that $x + v_1+..+v_k \ge 0$

In [ ]:
import numpy as np

# Create a mask for rows with negative values
mask = ~np.any(y < 0, axis=1)

# Filter out rows with negative values using the mask
filtered_y = y[mask]

filtered_y

So the whole neighborhood function becomes.

In [ ]:
def generate_search_neighborhood(schedule):
  N = sum(schedule)
  T = len(schedule)
  print(f'The schedule = {schedule}')
  
  # Generate a matrix 'v_star' using the 'get_v_star' function
  v_star = get_v_star(T)
  
  # Generate all possible non-empty subsets (powerset) of the set {0, 1, 2, ..., T-1}
  # 'ids' will be a list of tuples, where each tuple is a subset of indices
  ids = list(powerset(range(T)))

  # Select the vectors from 'v_star' that correspond to the indices in each subset
  # 'sub_sets' will be a list of lists, where each inner list contains vectors from 'v_star'
  sub_sets = [v_star[i] for i in ids]

  # Sum the vectors within each subset and flatten the result to get a 1-D array
  # 'summed_sets' will be a list of 1-D numpy arrays, where each array is the sum of vectors
  summed_sets = [np.sum(sub_sets[i], axis=0).flatten() for i in range(len(sub_sets))]
  
  neighborhood = np.array([schedule + summed_sets[i] for i in range(len(summed_sets))])
  
  # Create a mask for rows with negative values
  mask = ~np.any(neighborhood < 0, axis=1)

  # Filter out rows with negative values using the mask
  filtered_neighborhood = neighborhood[mask]
  print(f'And the neighborhood is {filtered_neighborhood}')
  return filtered_neighborhood

testnh = generate_search_neighborhood(x)

Let the objective function be a simple sum of square.

In [ ]:
def obj_function(x):
  res = sum(map(lambda i: i * i, x))
  return res

obj_function(np.array(x))

The algorithm for computing an optimal schedule becomes.

In [ ]:
def search_best_solution(x):
    obj_value = obj_function(x)
    
    # Keep track of the best solution found
    best_solution = x.copy()
    
    # Continue the search until no improvement is found
    while True:  # Start an outer loop that will continue until explicitly broken
        nh = generate_search_neighborhood(best_solution)  # Generate a new neighborhood
        improved = False  # Flag to check if an improvement was found in the inner loop
        
        for y in nh:  # Inner loop to search through the neighborhood
            c = obj_function(y)
            if c < obj_value:
                obj_value = c
                best_solution = y.copy()
                print(f'Found better schedule {best_solution} with objective value {obj_value}')
                improved = True  # Set the flag because an improvement was found
                break  # Exit the inner loop to generate a new neighborhood
        
        if not improved:  # If no improvement was found in the inner loop
            break  # Exit the outer loop - the search is complete

    return best_solution, obj_value

# Example usage:
x = [5, 0, 0, 0, 1]
initial_solution = x  # Replace this with your actual initial solution
best_solution, best_obj_value = search_best_solution(initial_solution)
print("Best solution found:", best_solution)
print("Objective value of the best solution:", best_obj_value)

In [ ]:
def search_solutions_3d(s, obj_func):
    """
    Perform a search for the best solution in a solution space using a neighborhood-based search algorithm.

    Args:
    s (list): The initial solution from which the search starts. Needs three elements x, y, z
    obj_func (function): The objective function that evaluates the quality of a solution.

    Returns:
    list: The x, y, z coordinates (or components) of each solution in the search history.
    list: The corresponding objective function values for each solution in the search history.

    The function initializes with an initial solution 's' and evaluates it using the objective function 'obj_func'.
    It then iteratively generates neighboring solutions, evaluates them, and keeps track of the best solution found so far.
    The search continues until no further improvement is found in the neighborhood of the current best solution.
    """

    # Evaluate the initial solution
    obj_value = obj_func(s)
    
    # Initialize the best solution as the initial solution
    best_solution = s.copy()
    
    # Initialize lists to keep track of the search history
    x, y, z, f_v = [], [], [], []
    
    # Start an outer loop that continues until no improvement is found
    while True:
        # Generate a new neighborhood of solutions around the current best solution
        nh = generate_search_neighborhood(best_solution)
        
        # Flag to check if an improvement was found in the current iteration
        improved = False
        
        # Iterate over each solution in the neighborhood
        for s in nh:
            # Append the components of the solution to the history lists
            x.append(s[0])
            y.append(s[1])
            z.append(s[2])
            
            # Evaluate the current solution
            c = obj_func(s)
            f_v.append(c)
            print(f'Tested solution {s} with objective value {c}')
            
            # Check if the current solution is an improvement
            if c < obj_value:
                # Update the best solution and its objective value
                obj_value = c
                best_solution = s.copy()
                print(f'Found better solution {best_solution} with objective value {obj_value}')
                
                # Mark that an improvement was found and exit the inner loop
                improved = True
                break
        
        # If no improvement was found in the entire neighborhood
        if not improved:
            # Exit the outer loop - the search is complete
            break

    # Return the search history
    return x, y, z, f_v

In [ ]:
s = [3, 0, 0]
x, y, z, f_v = search_solutions_3d(s, obj_function)

# Determine the range for each axis to set the ticks
s_range = range(int(min(s)), int(max(s)) + 1)

fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=6,
        color=f_v,  # Set color to function values
        colorscale='Plotly3',
        opacity=0.8,
        showscale=True
    ),
    hoverinfo='text',
    text=[f'x: {xi:.0f}, y: {yi:.0f}, z: {zi:.0f}, f_v: {value:.1f}' for xi, yi, zi, value in zip(x, y, z, f_v)]
)])

fig.update_layout(title='3D Function Visualization',
                  scene=dict(
                      xaxis=dict(title='X axis', tickmode='array', tickvals=list(s_range)),
                      yaxis=dict(title='Y axis', tickmode='array', tickvals=list(s_range)),
                      zaxis=dict(title='Z axis', tickmode='array', tickvals=list(s_range))
                  ))

fig.show()